In [268]:
# Impoartación de librerías
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from sklearn.feature_extraction.text import CountVectorizer
import unicodedata

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mpvac\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mpvac\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [269]:
# Lectura del archivo csv
movies_eda = pd.read_csv(r'C:\Users\mpvac\OneDrive\Documents\Mis Repositorios\Proyecto ML SH\venvpiml\dataset_api.csv', low_memory=False)
credits_eda = pd.read_csv(r'C:\Users\mpvac\OneDrive\Documents\Mis Repositorios\LABS\Proyecto 1\credits.csv', low_memory=False)

# convierto ID´s en entero
movies_eda['id'] = movies_eda['id'].astype('int')
credits_eda['id'] = credits_eda['id'].astype('int')

# merge de ambos dataframes
df_eda = pd.merge(movies_eda, credits_eda, on='id')

In [270]:
# Me quedo solo con las peliculas cuyo status sea estrenada
df_eda = df_eda[df_eda['status'] == 'Released']

In [271]:
# reviso porcentaje de datos nulos
df_eda.isnull().sum()/len(df_eda)*100

budget                    0.000000
id                        0.000000
original_language         0.022142
overview                  2.008281
popularity                0.000000
release_date              0.000000
revenue                   0.000000
runtime                   0.531408
status                    0.000000
tagline                  54.887851
title                     0.000000
vote_average              0.000000
vote_count                0.000000
belongs_to_collection    90.051591
genres                    5.156876
production_companies     25.735669
production_countries     13.522131
spoken_languages          8.227974
release_year              0.000000
return                    0.000000
director_name             1.791289
cast                      0.000000
crew                      0.000000
dtype: float64

In [272]:
# Elimino columnas que no voy a utilizar y columnas con muchos datos nulos
df_eda.drop(['budget', 'production_countries','production_companies', 'tagline', 'original_language', 'release_date', 'runtime', 'spoken_languages', 'status', 'belongs_to_collection'], axis=1, inplace=True)


In [273]:
# Elimino los datos nulos de las columnas que voy a utilizar
df_eda.dropna(inplace=True)

In [274]:
# elimino duplicados
df_eda.drop_duplicates(inplace=True)
df_eda.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41436 entries, 0 to 45606
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             41436 non-null  int32  
 1   overview       41436 non-null  object 
 2   popularity     41436 non-null  float64
 3   revenue        41436 non-null  int64  
 4   title          41436 non-null  object 
 5   vote_average   41436 non-null  float64
 6   vote_count     41436 non-null  float64
 7   genres         41436 non-null  object 
 8   release_year   41436 non-null  int64  
 9   return         41436 non-null  float64
 10  director_name  41436 non-null  object 
 11  cast           41436 non-null  object 
 12  crew           41436 non-null  object 
dtypes: float64(4), int32(1), int64(2), object(6)
memory usage: 4.3+ MB


In [275]:
# normalizo las columnas
df_eda['genres'] = df_eda['genres'].apply(literal_eval)
df_eda['cast'] = df_eda['cast'].apply(literal_eval)
df_eda['crew'] = df_eda['crew'].apply(literal_eval)

df_eda.head()

,id,overview,popularity,revenue,title,vote_average,vote_count,genres,release_year,return,director_name,cast,crew
0,862,"Led by Woody, Andy's toys live happily in his ...",21.946943,373554033,Toy Story,7.7,5415.0,"[Animation, Comedy, Family]",1995,12.451801,John Lasseter,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,8844,When siblings Judy and Peter discover an encha...,17.015539,262797249,Jumanji,6.9,2413.0,"[Adventure, Fantasy, Family]",1995,4.043035,Joe Johnston,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,15602,A family wedding reignites the ancient feud be...,11.712900,0,Grumpier Old Men,6.5,92.0,"[Romance, Comedy]",1995,0.000000,Howard Deutch,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,31357,"Cheated on, mistreated and stepped on, the wom...",3.859495,81452156,Waiting to Exhale,6.1,34.0,"[Comedy, Drama, Romance]",1995,5.090760,Forest Whitaker,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,11862,Just when George Banks has recovered from his ...,8.387519,76578911,Father of the Bride Part II,5.7,173.0,[Comedy],1995,0.000000,Charles Shyer,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."


In [276]:
# Veo el contenido de la columna cast
df_eda['cast'][0]

[{'cast_id': 14,
  'character': 'Woody (voice)',
  'credit_id': '52fe4284c3a36847f8024f95',
  'gender': 2,
  'id': 31,
  'name': 'Tom Hanks',
  'order': 0,
  'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'},
 {'cast_id': 15,
  'character': 'Buzz Lightyear (voice)',
  'credit_id': '52fe4284c3a36847f8024f99',
  'gender': 2,
  'id': 12898,
  'name': 'Tim Allen',
  'order': 1,
  'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'},
 {'cast_id': 16,
  'character': 'Mr. Potato Head (voice)',
  'credit_id': '52fe4284c3a36847f8024f9d',
  'gender': 2,
  'id': 7167,
  'name': 'Don Rickles',
  'order': 2,
  'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'},
 {'cast_id': 17,
  'character': 'Slinky Dog (voice)',
  'credit_id': '52fe4284c3a36847f8024fa1',
  'gender': 2,
  'id': 12899,
  'name': 'Jim Varney',
  'order': 3,
  'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'},
 {'cast_id': 18,
  'character': 'Rex (voice)',
  'credit_id': '52fe4284c3a36847f8024fa5',
  'gender': 2,
  'id': 12900,
 

In [277]:
# obtengo el nombre del actor principal
df_eda['protagonista'] = df_eda['cast'].apply(lambda x: [i['name'] for i in x if i['order'] == 0])
df_eda.head()


,id,overview,popularity,revenue,title,vote_average,vote_count,genres,release_year,return,director_name,cast,crew,protagonista
0,862,"Led by Woody, Andy's toys live happily in his ...",21.946943,373554033,Toy Story,7.7,5415.0,"[Animation, Comedy, Family]",1995,12.451801,John Lasseter,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",[Tom Hanks]
1,8844,When siblings Judy and Peter discover an encha...,17.015539,262797249,Jumanji,6.9,2413.0,"[Adventure, Fantasy, Family]",1995,4.043035,Joe Johnston,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",[Robin Williams]
2,15602,A family wedding reignites the ancient feud be...,11.712900,0,Grumpier Old Men,6.5,92.0,"[Romance, Comedy]",1995,0.000000,Howard Deutch,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",[Walter Matthau]
3,31357,"Cheated on, mistreated and stepped on, the wom...",3.859495,81452156,Waiting to Exhale,6.1,34.0,"[Comedy, Drama, Romance]",1995,5.090760,Forest Whitaker,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",[Whitney Houston]
4,11862,Just when George Banks has recovered from his ...,8.387519,76578911,Father of the Bride Part II,5.7,173.0,[Comedy],1995,0.000000,Charles Shyer,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",[Steve Martin]


In [278]:
# Veo el valor de la columna crew  
df_eda['crew'][1]

[{'credit_id': '52fe44bfc3a36847f80a7cd1',
  'department': 'Production',
  'gender': 2,
  'id': 511,
  'job': 'Executive Producer',
  'name': 'Larry J. Franco',
  'profile_path': None},
 {'credit_id': '52fe44bfc3a36847f80a7c89',
  'department': 'Writing',
  'gender': 2,
  'id': 876,
  'job': 'Screenplay',
  'name': 'Jonathan Hensleigh',
  'profile_path': '/l1c4UFD3g0HVWj5f0CxXAvMAGiT.jpg'},
 {'credit_id': '52fe44bfc3a36847f80a7cdd',
  'department': 'Sound',
  'gender': 2,
  'id': 1729,
  'job': 'Original Music Composer',
  'name': 'James Horner',
  'profile_path': '/oLOtXxXsYk8X4qq0ud4xVypXudi.jpg'},
 {'credit_id': '52fe44bfc3a36847f80a7c7d',
  'department': 'Directing',
  'gender': 2,
  'id': 4945,
  'job': 'Director',
  'name': 'Joe Johnston',
  'profile_path': '/fok4jaO62v5IP6hkpaaAcXuw2H.jpg'},
 {'credit_id': '52fe44bfc3a36847f80a7cd7',
  'department': 'Editing',
  'gender': 2,
  'id': 4951,
  'job': 'Editor',
  'name': 'Robert Dalva',
  'profile_path': None},
 {'credit_id': '57352

In [279]:
# obtengo el nombre de los empleados del sector escritores
df_eda['crew_writing'] = df_eda['crew'].apply(lambda x: [i['name'] for i in x if i.get('department', 0) == 'Writing'] if isinstance(x, list) else [])

In [280]:
df_eda.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41436 entries, 0 to 45606
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             41436 non-null  int32  
 1   overview       41436 non-null  object 
 2   popularity     41436 non-null  float64
 3   revenue        41436 non-null  int64  
 4   title          41436 non-null  object 
 5   vote_average   41436 non-null  float64
 6   vote_count     41436 non-null  float64
 7   genres         41436 non-null  object 
 8   release_year   41436 non-null  int64  
 9   return         41436 non-null  float64
 10  director_name  41436 non-null  object 
 11  cast           41436 non-null  object 
 12  crew           41436 non-null  object 
 13  protagonista   41436 non-null  object 
 14  crew_writing   41436 non-null  object 
dtypes: float64(4), int32(1), int64(2), object(8)
memory usage: 5.9+ MB


In [281]:
df_eda.head()

,id,overview,popularity,revenue,title,vote_average,vote_count,genres,release_year,return,director_name,cast,crew,protagonista,crew_writing
0,862,"Led by Woody, Andy's toys live happily in his ...",21.946943,373554033,Toy Story,7.7,5415.0,"[Animation, Comedy, Family]",1995,12.451801,John Lasseter,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",[Tom Hanks],"[Joss Whedon, Andrew Stanton, Joel Cohen, Alec..."
1,8844,When siblings Judy and Peter discover an encha...,17.015539,262797249,Jumanji,6.9,2413.0,"[Adventure, Fantasy, Family]",1995,4.043035,Joe Johnston,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",[Robin Williams],"[Jonathan Hensleigh, Chris van Allsburg, Greg ..."
2,15602,A family wedding reignites the ancient feud be...,11.712900,0,Grumpier Old Men,6.5,92.0,"[Romance, Comedy]",1995,0.000000,Howard Deutch,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",[Walter Matthau],"[Mark Steven Johnson, Mark Steven Johnson]"
3,31357,"Cheated on, mistreated and stepped on, the wom...",3.859495,81452156,Waiting to Exhale,6.1,34.0,"[Comedy, Drama, Romance]",1995,5.090760,Forest Whitaker,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",[Whitney Houston],"[Ronald Bass, Terry McMillan, Terry McMillan]"
4,11862,Just when George Banks has recovered from his ...,8.387519,76578911,Father of the Bride Part II,5.7,173.0,[Comedy],1995,0.000000,Charles Shyer,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",[Steve Martin],"[Nancy Meyers, Albert Hackett]"


In [282]:
# elimino columnas que no voy a utilizar
df_eda.drop(['cast', 'crew'], axis=1, inplace=True)
df_eda.head()

,id,overview,popularity,revenue,title,vote_average,vote_count,genres,release_year,return,director_name,protagonista,crew_writing
0,862,"Led by Woody, Andy's toys live happily in his ...",21.946943,373554033,Toy Story,7.7,5415.0,"[Animation, Comedy, Family]",1995,12.451801,John Lasseter,[Tom Hanks],"[Joss Whedon, Andrew Stanton, Joel Cohen, Alec..."
1,8844,When siblings Judy and Peter discover an encha...,17.015539,262797249,Jumanji,6.9,2413.0,"[Adventure, Fantasy, Family]",1995,4.043035,Joe Johnston,[Robin Williams],"[Jonathan Hensleigh, Chris van Allsburg, Greg ..."
2,15602,A family wedding reignites the ancient feud be...,11.712900,0,Grumpier Old Men,6.5,92.0,"[Romance, Comedy]",1995,0.000000,Howard Deutch,[Walter Matthau],"[Mark Steven Johnson, Mark Steven Johnson]"
3,31357,"Cheated on, mistreated and stepped on, the wom...",3.859495,81452156,Waiting to Exhale,6.1,34.0,"[Comedy, Drama, Romance]",1995,5.090760,Forest Whitaker,[Whitney Houston],"[Ronald Bass, Terry McMillan, Terry McMillan]"
4,11862,Just when George Banks has recovered from his ...,8.387519,76578911,Father of the Bride Part II,5.7,173.0,[Comedy],1995,0.000000,Charles Shyer,[Steve Martin],"[Nancy Meyers, Albert Hackett]"


In [283]:
# Aplico lower 
df_eda['genres'] = df_eda['genres'].apply(lambda x: [genre.lower() for genre in x])
df_eda['director_name'] = df_eda['director_name'].str.lower()
df_eda['protagonista'] = df_eda['protagonista'].apply(lambda x: [genre.lower() for genre in x])
df_eda.head()


,id,overview,popularity,revenue,title,vote_average,vote_count,genres,release_year,return,director_name,protagonista,crew_writing
0,862,"Led by Woody, Andy's toys live happily in his ...",21.946943,373554033,Toy Story,7.7,5415.0,"[animation, comedy, family]",1995,12.451801,john lasseter,[tom hanks],"[Joss Whedon, Andrew Stanton, Joel Cohen, Alec..."
1,8844,When siblings Judy and Peter discover an encha...,17.015539,262797249,Jumanji,6.9,2413.0,"[adventure, fantasy, family]",1995,4.043035,joe johnston,[robin williams],"[Jonathan Hensleigh, Chris van Allsburg, Greg ..."
2,15602,A family wedding reignites the ancient feud be...,11.712900,0,Grumpier Old Men,6.5,92.0,"[romance, comedy]",1995,0.000000,howard deutch,[walter matthau],"[Mark Steven Johnson, Mark Steven Johnson]"
3,31357,"Cheated on, mistreated and stepped on, the wom...",3.859495,81452156,Waiting to Exhale,6.1,34.0,"[comedy, drama, romance]",1995,5.090760,forest whitaker,[whitney houston],"[Ronald Bass, Terry McMillan, Terry McMillan]"
4,11862,Just when George Banks has recovered from his ...,8.387519,76578911,Father of the Bride Part II,5.7,173.0,[comedy],1995,0.000000,charles shyer,[steve martin],"[Nancy Meyers, Albert Hackett]"


In [284]:
# quito los espacios a columnas que voy a utilizar
df_eda['protagonista'] = df_eda['protagonista'].apply(lambda x: [genre.replace(" ", "") for genre in x])
df_eda["genres"] = df_eda["genres"].apply(lambda x: [genre.replace(" ", "") for genre in x])
df_eda["director_name"] = df_eda["director_name"].str.replace(" ", "")
df_eda.head()

,id,overview,popularity,revenue,title,vote_average,vote_count,genres,release_year,return,director_name,protagonista,crew_writing
0,862,"Led by Woody, Andy's toys live happily in his ...",21.946943,373554033,Toy Story,7.7,5415.0,"[animation, comedy, family]",1995,12.451801,johnlasseter,[tomhanks],"[Joss Whedon, Andrew Stanton, Joel Cohen, Alec..."
1,8844,When siblings Judy and Peter discover an encha...,17.015539,262797249,Jumanji,6.9,2413.0,"[adventure, fantasy, family]",1995,4.043035,joejohnston,[robinwilliams],"[Jonathan Hensleigh, Chris van Allsburg, Greg ..."
2,15602,A family wedding reignites the ancient feud be...,11.712900,0,Grumpier Old Men,6.5,92.0,"[romance, comedy]",1995,0.000000,howarddeutch,[waltermatthau],"[Mark Steven Johnson, Mark Steven Johnson]"
3,31357,"Cheated on, mistreated and stepped on, the wom...",3.859495,81452156,Waiting to Exhale,6.1,34.0,"[comedy, drama, romance]",1995,5.090760,forestwhitaker,[whitneyhouston],"[Ronald Bass, Terry McMillan, Terry McMillan]"
4,11862,Just when George Banks has recovered from his ...,8.387519,76578911,Father of the Bride Part II,5.7,173.0,[comedy],1995,0.000000,charlesshyer,[stevemartin],"[Nancy Meyers, Albert Hackett]"


In [285]:
df_eda.reset_index(drop=True, inplace=True)


In [286]:
# get dummies de la columna genres
df_eda = df_eda.join(df_eda.genres.str.join('|').str.get_dummies())
df_eda.head()

,id,overview,popularity,revenue,title,vote_average,vote_count,genres,release_year,return,...,history,horror,music,mystery,romance,sciencefiction,thriller,tvmovie,war,western
0,862,"Led by Woody, Andy's toys live happily in his ...",21.946943,373554033,Toy Story,7.7,5415.0,"[animation, comedy, family]",1995,12.451801,...,0,0,0,0,0,0,0,0,0,0
1,8844,When siblings Judy and Peter discover an encha...,17.015539,262797249,Jumanji,6.9,2413.0,"[adventure, fantasy, family]",1995,4.043035,...,0,0,0,0,0,0,0,0,0,0
2,15602,A family wedding reignites the ancient feud be...,11.712900,0,Grumpier Old Men,6.5,92.0,"[romance, comedy]",1995,0.000000,...,0,0,0,0,1,0,0,0,0,0
3,31357,"Cheated on, mistreated and stepped on, the wom...",3.859495,81452156,Waiting to Exhale,6.1,34.0,"[comedy, drama, romance]",1995,5.090760,...,0,0,0,0,1,0,0,0,0,0
4,11862,Just when George Banks has recovered from his ...,8.387519,76578911,Father of the Bride Part II,5.7,173.0,[comedy],1995,0.000000,...,0,0,0,0,0,0,0,0,0,0


In [287]:
# genero df con las columnas con datos numericos
df_eda_numerico = df_eda.select_dtypes(include=['int64', 'float64'])

df_eda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41436 entries, 0 to 41435
Data columns (total 33 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              41436 non-null  int32  
 1   overview        41436 non-null  object 
 2   popularity      41436 non-null  float64
 3   revenue         41436 non-null  int64  
 4   title           41436 non-null  object 
 5   vote_average    41436 non-null  float64
 6   vote_count      41436 non-null  float64
 7   genres          41436 non-null  object 
 8   release_year    41436 non-null  int64  
 9   return          41436 non-null  float64
 10  director_name   41436 non-null  object 
 11  protagonista    41436 non-null  object 
 12  crew_writing    41436 non-null  object 
 13  action          41436 non-null  int64  
 14  adventure       41436 non-null  int64  
 15  animation       41436 non-null  int64  
 16  comedy          41436 non-null  int64  
 17  crime           41436 non-null 

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity

# Obtener las columnas relevantes
columnas = ['genres', 'protagonista']

# Crear un objeto MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Aplicar MultiLabelBinarizer a las columnas
features = mlb.fit_transform(df_eda[columnas])

entrada = 'Matrix'

# Calcular la similitud del coseno entre la entrada y el resto de las películas
entrada_index = df_eda[df_eda['title'] == entrada].index[0]  # Obtener el índice de la entrada
entrada_features = features[entrada_index].reshape(1, -1)  # Obtener las características de la entrada

similares = cosine_similarity(entrada_features, features)

# Obtener los índices de las películas más similares
top5similares = similares.argsort()[0][-5:]

# Obtener los títulos recomendados
recomendacion = df_eda.loc[top5similares, 'title'].tolist()

print("Las 5 películas más similares a", entrada, "son:")
for titulo in recomendacion:
    print(titulo)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

columnas = ['genres', 'cast_top3', 'crew_writing']

# Combinar las columnas en una sola columna de texto
df_eda['combined_features'] = df_eda[columnas].apply(lambda x: ' '.join(x), axis=1)

# Crear la matriz de características utilizando CountVectorizer
vectorizer = CountVectorizer()
matrix = vectorizer.fit_transform(df_eda['combined_features'])

# Calcular la similitud del coseno entre la entrada y el resto de las películas
entrada = "Matrix"
f_entrada = vectorizer.transform([entrada])
similares = cosine_similarity(f_entrada, matrix)

# Obtener los índices de las películas más similares
top5similares = similares.argsort()[0][-5:]

# Obtener los títulos recomendados
recomendacion = df_eda.loc[top5similares, 'title'].tolist()

print("Las 5 películas más similares a Toy Story son:")
for titulo in recomendacion:
    print(titulo)


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
# Crear una instancia de MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Convertir la lista de cadenas en representación binaria
binary_values = mlb.fit_transform(df_eda['cast_top5'])

# Crear un nuevo DataFrame con las nuevas columnas
binary_df = pd.DataFrame(binary_values, columns=mlb.classes_)

binary_df.head()

In [ ]:
binary_df.info()

In [ ]:
df_eda = df_eda.join(df_eda["cast_top5"].str.join('|').str.get_dummies())

In [ ]:
# genero un simplificador para quitar palabras comunes del lenguaje ingles
simplificador = set(stopwords.words('english')) 

# Función para limpiar el texto
def clean_text(text):
    text = text.lower() # Converto a minúsculas
    text = text.translate(str.maketrans('', '', string.punctuation)) # Elimino puntuación
    tokens = word_tokenize(text) # Tokenizar
    tokens = [word for word in tokens if word not in simplificador] # Elimino stopwords
    text = ' '.join(tokens) # Unir tokens
    return text

In [ ]:
# Aplico la función
df_eda['overview_simple'] = df_eda['overview'].apply(clean_text) 

In [ ]:
# elimino los números
df_eda['overview_simple'] = df_eda['overview_simple'].str.replace('\d+', '') 

In [ ]:
#elimino columnas que no voy a utilizar
df_eda.drop(['overview'], axis=1, inplace=True)

df_eda.head()

In [ ]:
# Funcion para eliminar palabras duplicadas en una misma fila de la columna overview_simple 
def remove_duplicate_words(row):
    words = row.split()  # Dividir el texto en palabras
    words_unique = list(set(words))  # Convertir a conjunto para eliminar duplicados y luego volver a lista
    
    return ' '.join(words_unique)  # Unir las palabras nuevamente en un solo string

# Aplicar la función a la columna 'overview_simple'
df_eda['overview_simple'] = df_eda['overview_simple'].apply(remove_duplicate_words)

df_eda.head()




In [ ]:
# ellimino palabras con menos de 5 caracteres en la columna overview_simple
df_eda['overview_simple'] = df_eda['overview_simple'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>5]))

df_eda.head()

In [ ]:
datos_modelo = df_eda['overview_simple']
from sklearn.feature_extraction.text import  TfidfVectorizer
from sklearn.feature_extraction.text import  CountVectorizer


# selecciono 10000 filas al azar de la columna overview_simple para realizar el modelo
datos_modelo = df_eda.sample(10000, random_state=42)

# Creo el vectorizador
vectorizador = TfidfVectorizer(max_features=1000, stop_words='english') 

# Creo la matriz de términos
matrix = vectorizador.fit_transform(datos_modelo['overview_simple'])



In [ ]:
# Obtengo los índices de las películas
indices = pd.Series(df_eda.index, index=df_eda['title']).drop_duplicates()

# función para obtener recomendaciones
def get_recommendations(title, cosine_sim=matrix):
    # Obtengo el índice de la película que coincide con el título
    idx = indices[title]
    
    # Obtengo las puntuaciones de similitud de coseno de todas las películas con esa película
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Ordeno las películas en función de las puntuaciones de similitud de coseno
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Obtengo las puntuaciones de las 10 películas más similares
    sim_scores = sim_scores[1:11]
    
    # Obtengo los índices de las películas
    movie_indices = [i[0] for i in sim_scores]
    
    # Devuelvo las 10 películas más similares
    return df_eda['title'].iloc[movie_indices]

In [ ]:

# Obtengo las recomendaciones para la película 'The Dark Knight Rises'
get_recommendations('The Dark Knight Rises')

In [ ]:
# creo Vectorizer eliminando las palabras que aparecen en el 85% de los documentos y las que aparecen en menos de 5 documentos
vectorizer = TfidfVectorizer(max_df=0.75, min_df=5)


In [ ]:
# Ajusto y transformo los datos de la columna de texto
datos_modelo = vectorizer.fit_transform(df_eda['overview_simple'])

In [ ]:
from sklearn.decomposition import TruncatedSVD

# Aplicar TruncatedSVD para reducir la dimensionalidad
n_components = 100  # (componentes principales)
svd = TruncatedSVD(n_components=n_components)
datos_modelo_reducido = svd.fit_transform(datos_modelo)


In [ ]:
# Creo dataframe con los datos reducidos
datos_modelo_reducido = pd.DataFrame(datos_modelo_reducido, columns=[f'component_{i+1}' for i in range(n_components)])

datos_modelo_reducido.head()   

In [ ]:
# función para obtener recomendaciones
def get_recommendations(title, cosine_sim=cosine_sim):
    # Obtener el índice de la película que coincide con el título
    idx = df_eda[df_eda['title'] == title].index[0]

    # Obtener las puntuaciones de similitud de pares ordenados con todas las películas
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ordenar las películas en función de las puntuaciones de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtener las puntuaciones de las 10 películas más similares
    sim_scores = sim_scores[1:11]

    # Obtener los índices de las películas
    movie_indices = [i[0] for i in sim_scores]

    # Devolver las 10 películas más similares
    return df_eda['title'].iloc[movie_indices]

# Prueba de la función
get_recommendations('Toy Story')